In [1]:
# Library imports
import pandas as pd
import networkx as nx
import numpy as np
from NetworkGraph import NetworkGraph

In [2]:
df=pd.read_csv("Output\\emails_all_bck.csv")

df.shape

(2811799, 2)

In [3]:
df2 = df.groupby(['From','To']).size().reset_index()

df2.columns = ['From', 'To', 'Weight']

df2.head()

,From,To,Weight
0,'todd'.delahoussaye@enron.com,'todd'.delahoussaye@enron.com,5
1,'todd'.delahoussaye@enron.com,ajay.sharma@enron.com,5
2,'todd'.delahoussaye@enron.com,anne.bike@enron.com,1
3,'todd'.delahoussaye@enron.com,bianca.ornelas@enron.com,5
4,'todd'.delahoussaye@enron.com,brant.reves@enron.com,5


The NetworkGraph class takes in a list of tuples containing the edges - as such we need to convert the grouped data frame before generating the graph object

In [4]:
edges = [(x[0], x[1], x[2]) for x in list(df2.values)]

In [5]:
G = NetworkGraph(edges)

G.graph.number_of_nodes()

71971

In [6]:
G.graph.number_of_edges()

288695

Using the implemented cycle finder, identify the cycles within the graph

In [7]:
#G.findCycles()

#G.printCycles()

# THis currently has a recursion depth error

In [8]:
nx.number_connected_components(G.graph.to_undirected())

1124

This is a surprisingly high number of connected components and is most likely down to the witheld data - some of these gaps should be filled in with the remaining data so that the number of components decreases. Regardless, the focus should be on individual components - we can focus on those with features / individuals of interest.
Lets first see how many nodes/edges each component has and then identify cycles / cliques withtin them

In [9]:
subgraphs = nx.weakly_connected_component_subgraphs(G.graph, copy=True)

In [10]:
components = []

for g in subgraphs:
    params = (g.number_of_edges(), g.number_of_nodes())
    components.append(params)
    
components.sort(reverse=True)
components[:10]

[(287014, 69476),
 (100, 101),
 (42, 43),
 (28, 29),
 (22, 23),
 (21, 14),
 (14, 15),
 (13, 14),
 (12, 13),
 (10, 11)]

It looks like most of the graph is contained in a single large component with the remaining pieces small components with just a few nodes. Lets try and find any cycles in any of the smaller ones

In [11]:
subgraphs = nx.weakly_connected_component_subgraphs(G.graph, copy=True)

for g in subgraphs:
    if g.number_of_edges() < 2000:
        net = NetworkGraph(g)
        net.findCycles()
        if len(net.cycles) > 0:
            net.printCycles()

merlyn@stonehenge.com->cp@onsitetech.com->tex@off.org->merlyn@stonehenge.com
merlyn@stonehenge.com->cp@onsitetech.com->merlyn@stonehenge.com
merlyn@stonehenge.com->cp@onsitetech.com->tex@off.org->merlyn@stonehenge.com
merlyn@stonehenge.com->cp@onsitetech.com->merlyn@stonehenge.com


There doesn't look to be anything of interest in the small components so we can probably discard them when doing the analysis